# 1.CNN(합성곱 신경망 네트워크) 소개

컨브넷의 정의와 컨브넷이 컴퓨터 비전 관련 작업에 잘 맞는 이유에 대해 이론적 배경을 알아보겠습니다. 하지만 먼저 간단한 컨브넷 예제를 둘러 보죠. 앞선 실습에서 완전 연결 네트워크로 풀었던(이 방식의 테스트 정확도는 97.8%였습니다) MNIST 숫자 이미지 분류에 컨브넷을 사용해 보겠습니다. 기본적인 컨브넷이더라도 완전 연결된 모델의 성능을 훨씬 앞지를 것입니다.

다음 코드는 기본적인 컨브넷의 모습입니다. Conv2D와 MaxPooling2D 층을 쌓아 올렸습니다. 잠시 후에 이들이 무엇인지 배우겠습니다.

컨브넷이 (image_height, image_width, image_channels) 크기의 입력 텐서를 사용한다는 점이 중요합니다(배치 차원은 포함하지 않습니다). 이 예제에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하도록 컨브넷을 설정해야 합니다. 이 때문에 첫 번째 층의 매개변수로 input_shape=(28, 28, 1)을 전달합니다.

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# 2.CNN 기본 컨브넷 구조 출력

In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels) 크기의 3D 텐서입니다. 높이와 넓이 차원은 네트워크가 깊어질수록 작아지는 경향이 있습니다. 채널의 수는 Conv2D 층에 전달된 첫 번째 매개변수에 의해 조절됩니다(32개 또는 64개).

다음 단계에서 마지막 층의 ((3, 3, 64) 크기인) 출력 텐서를 완전 연결 네트워크에 주입합니다. 이 네트워크는 이미 익숙하게 보았던 Dense 층을 쌓은 분류기입니다. 이 분류기는 1D 벡터를 처리하는데 이전 층의 출력이 3D 텐서입니다. 그래서 먼저 3D 출력을 1D 텐서로 펼쳐야 합니다. 그다음 몇 개의 Dense 층을 추가합니다:

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

10개의 클래스를 분류하기 위해 마지막 층의 출력 크기를 10으로 하고 소프트맥스 활성화 함수를 사용합니다. 이제 전체 네트워크는 다음과 같습니다:

In [ ]:
## 소프트맥스 활성화 함수 추가된 컨브넷 구조 출력

# 3.소프트맥스 활성화 함수 추가된 CNN 컨브넷 구조 출력

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)               

# 4.MNIST 데이터셋을 가져와서 CNN 훈련 데이터로 리쉐이핑

In [6]:

from keras.datasets import mnist
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11501568/11490434 [==============================] - 0s 0us/step


# 5.모델 최적화 및 모델 학습

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 70s 57ms/step - loss: 0.4062 - accuracy: 0.8694
Epoch 2/5
938/938 [==============================] - 54s 58ms/step - loss: 0.0503 - accuracy: 0.9843
Epoch 3/5
938/938 [==============================] - 54s 57ms/step - loss: 0.0326 - accuracy: 0.9891
Epoch 4/5
938/938 [==============================] - 53s 57ms/step - loss: 0.0215 - accuracy: 0.9934
Epoch 5/5
938/938 [==============================] - 53s 57ms/step - loss: 0.0185 - accuracy: 0.9940


테스트 데이터에서 모델을 평가해 보죠:

# 6.모델 성능 평가

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 4s 11ms/step - loss: 0.0305 - accuracy: 0.9904


# 7.학습된 CNN 모델 사용하여 MNIST 숫자 알아맞히기

In [ ]:
import numpy as np
# 완성된 모델을 사용해보자 (Predict)
x = test_images[0:2]               # 테스트의 첫 샘플을 가져와 사용해보자. 
#y = model.predict_classes(x)  # 기대값이 가장 높은 결과 값 얻음
y = np.argmax(model.predict(x), axis=-1) # 기대값이 가장 높은 결과 값 얻음
print("결과: ")
print(y)
z = model.predict(x)              # 각 클래스(0~9) 기대값 얻음
print("각 숫자에 대한 가능성:")
print(z)

결과: 
[7 2]
각 숫자에 대한 가능성:
[[6.6857639e-09 8.8880036e-08 3.3702401e-07 1.6979682e-07 1.6001761e-09
  2.8564568e-08 2.1178062e-14 9.9999881e-01 1.8883080e-09 5.5924608e-07]
 [2.6077357e-08 1.9872277e-06 9.9999797e-01 1.2313578e-13 3.3403713e-10
  4.1100012e-15 3.9038635e-08 1.0260870e-12 2.9322456e-09 6.0665035e-14]]
